<a href="https://colab.research.google.com/github/sakaxo/tensorflow_with_django_model_notebook/blob/master/animal_classification_note_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Import Project Dependancies***

In [32]:
import numpy as np
import random
import pickle
import cv2
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [33]:
data_dir = "/content/dataset"

In [34]:
class_names = ['cat','dog','parrot']
img_size = 100

In [35]:
data = []

# **Prepare Data set**

In [36]:
for animal in class_names:
  class_path = os.path.join(data_dir,animal)
  label = class_names.index(animal)
  for img in os.listdir(class_path):
    img_path = os.path.join(class_path,img)

    img_arr = cv2.imread(img_path,0) #load image and convert to gray scale (0)
    img_arr = cv2.resize(img_arr,(img_size,img_size)) # resize every image to size (img_size,img_size)

    # data
    data.append([img_arr,label])
  


In [37]:
random.shuffle(data)
X = []
y = []

In [38]:
for feature,label in data:
  X.append(feature)
  y.append(label)

**Save Prepared data to memory**

In [39]:
pickle.dump(X,open('train_X.pkl','wb'))
pickle.dump(y,open('train_y.pkl','wb'))

In [40]:
X_train = pickle.load(open('train_X.pkl','rb'))
y_train = pickle.load(open('train_y.pkl','rb'))


# **Data list to numpy array**

In [41]:
X = np.array(X_train)
y = np.array(y_train)

In [42]:
# scale features
X = X/255.0


# **Build and Train Model**

**Build model**

In [43]:
model_f = keras.Sequential(name="Sequential_1")
model_f.add(layers.Flatten(input_shape=(100,100), name="Flatten_layer"))
model_f.add(layers.Dense(32, activation="relu", name="layer_1"))
model_f.add(layers.Dense(32, activation="relu", name="layer_2"))
model_f.add(layers.Dense(3, activation="softmax", name="layer_3"))

model_f.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

**Train model**

In [44]:
model_f.fit(X, y, epochs=10,validation_split=0.3,)

Epoch 1/10
40/40 [==============================] - 1s 15ms/step - loss: 1.2109 - accuracy: 0.3738 - val_loss: 1.0848 - val_accuracy: 0.4074
Epoch 2/10
40/40 [==============================] - 0s 10ms/step - loss: 1.1090 - accuracy: 0.3952 - val_loss: 1.1041 - val_accuracy: 0.4241
Epoch 3/10
40/40 [==============================] - 0s 10ms/step - loss: 1.0778 - accuracy: 0.4302 - val_loss: 1.0878 - val_accuracy: 0.4148
Epoch 4/10
40/40 [==============================] - 0s 10ms/step - loss: 1.0401 - accuracy: 0.4730 - val_loss: 1.0491 - val_accuracy: 0.4333
Epoch 5/10
40/40 [==============================] - 0s 9ms/step - loss: 1.0249 - accuracy: 0.4897 - val_loss: 1.0921 - val_accuracy: 0.4315
Epoch 6/10
40/40 [==============================] - 0s 10ms/step - loss: 1.1014 - accuracy: 0.4468 - val_loss: 1.4286 - val_accuracy: 0.3481
Epoch 7/10
40/40 [==============================] - 0s 10ms/step - loss: 1.0463 - accuracy: 0.4619 - val_loss: 1.0942 - val_accuracy: 0.4130
Epoch 8/10
40/

**save built model**

In [45]:
model_f.save('animal_model.h5')